In [7]:
import pandas as pd
import numpy as np

### Семинар 1. Введение в NLP
Задача: классификация текстов.  
Идея: Посмотрим, какие слова встречаются наиболее часто, и будем определять по ним

In [35]:
df = pd.read_csv('./important_conversations.csv').fillna("")
df.head()

,Unnamed: 0,text,is_female
0,0,,True
1,1,Отлично!,False
2,2,https://telegram.me/joinchat/AjKh-0GHAsjVpK5Nw...,False
3,3,Драконы!,False
4,4,,False


#### Необходимые вещи из тервера
$P(A)$ - вероятность события A  
$P(A \cap B)$ - вероятность, что A и B   
$P(A|B)$ - вероятность события А при условии, что произошло B

$$P(A|B) = \frac{P(A \cap B)}{P(B)}$$

Формула Байеса

$$P(B|A) = \frac{P(A | B)P(B)}{P(A)}$$

Пример задачи:
По статистике, у 80% нормосных девушек есть молодой человек. При этом, среди всех девушек 60% нормосные и у 50% есть молодой человек. Ваш друг сказал, что познакомит вас с одинокой девушкой. Какова вероятность, что она будет нормосной?

#### Naive Bayes
$$P(Class = c|Message = d) = \frac{P(Message = d | Class = c)P(Class = c)}{P(Message = d)}$$
Соответственно, чтобы определить класс сообщения $d$, выбираем класс $c$ такой, чтобы $P(Class = c|Message = d)$ была максимальной. 
$$P(c|d) = \frac{P(d|c)P(c)}{P(d)} \propto P(d|c)P(c) \propto \prod P(w_i|c)P(c)$$
$P(w_i|c)$ - вероятность того, что в сообщении из класса $c$ встретится встретится слово $w_i$ 

In [9]:
import nltk

In [10]:
messages = [item.split() for item in df.text]
words_flatten = [w for message in messages for w in message]

In [11]:
all_words = nltk.FreqDist(w.lower() for w in words_flatten)

In [45]:
all_words.most_common(50)

[('в', 6205),
 ('не', 5495),
 ('и', 5182),
 ('на', 3259),
 ('что', 2970),
 ('я', 2888),
 ('а', 2712),
 ('это', 2384),
 ('у', 2162),
 ('с', 2024),
 ('как', 1697),
 ('там', 1390),
 ('по', 1325),
 ('ты', 1305),
 ('ну', 1283),
 ('так', 1224),
 ('-', 1144),
 ('но', 1139),
 ('он', 1019),
 ('все', 1012),
 ('то', 962),
 ('есть', 943),
 ('за', 871),
 ('же', 862),
 ('если', 859),
 ('из', 798),
 ('меня', 738),
 ('вот', 720),
 ('да', 710),
 ('просто', 694),
 ('уже', 690),
 ('для', 682),
 ('они', 676),
 ('только', 669),
 ('к', 653),
 ('мне', 642),
 ('нет', 641),
 ('или', 621),
 ('можно', 612),
 ('бы', 611),
 ('тут', 573),
 ('еще', 556),
 ('про', 555),
 ('от', 546),
 ('будет', 530),
 ('тоже', 530),
 ('нас', 508),
 ('надо', 490),
 ('его', 485),
 ('было', 484)]

In [40]:
word_features = [item[0] for item in all_words.most_common(500)]
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [42]:
featuresets = [(document_features(d), c) for (d,c) in zip(messages,df.is_female)]

In [43]:
train_set, test_set = featuresets[5000:], featuresets[:5000]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [44]:
print(nltk.classify.accuracy(classifier, test_set))

0.9396


In [65]:
classifier.show_most_informative_features(50) 

Most Informative Features
            contains(:)) = True            False : True   =      6.7 : 1.0
         contains(чтобы) = True            False : True   =      6.1 : 1.0
        contains(ребят,) = True             True : False  =      5.9 : 1.0
             contains(7) = True             True : False  =      5.0 : 1.0
             contains(>) = True            False : True   =      4.8 : 1.0
          contains(чтоб) = True            False : True   =      4.7 : 1.0
            contains(её) = True            False : True   =      4.7 : 1.0
        contains(питере) = True             True : False  =      4.6 : 1.0
         contains(итоге) = True             True : False  =      4.6 : 1.0
       contains(климчик) = True             True : False  =      4.6 : 1.0
        contains(россии) = True             True : False  =      4.4 : 1.0
          contains(олег) = True             True : False  =      4.4 : 1.0
          contains(ага,) = True             True : False  =      4.3 : 1.0

#### Стемминг и лемматизация
Лемматизация - разделение предложения на термы (слова, знаки препинания и т.д)
Стемминг - приведение слов в начальную форму

In [110]:
from nltk import SnowballStemmer
stemmer = SnowballStemmer('english')

In [119]:
stemmer.stem('chips')

'chip'

In [104]:
from pymystem3 import Mystem
mystem = Mystem()

In [105]:
mystem.analyze('через бота мне что-то про Зимбабве выдает, а на сайте про синдром Куниса')
#Расшифровки: https://tech.yandex.ru/mystem/doc/grammemes-values-docpage/

[{'analysis': [{'gr': 'PR=', 'lex': 'через'}], 'text': 'через'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,муж,неод=род,ед', 'lex': 'бот'}], 'text': 'бота'},
 {'text': ' '},
 {'analysis': [{'gr': 'SPRO,ед,1-л=(пр|дат)', 'lex': 'я'}], 'text': 'мне'},
 {'text': ' '},
 {'analysis': [{'gr': 'SPRO,ед,сред,неод=(вин|им)', 'lex': 'что-то'}],
  'text': 'что-то'},
 {'text': ' '},
 {'analysis': [{'gr': 'PR=', 'lex': 'про'}], 'text': 'про'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,сред,неод=(пр,мн|пр,ед|вин,мн|вин,ед|дат,мн|дат,ед|род,мн|род,ед|твор,мн|твор,ед|им,мн|им,ед)',
    'lex': 'зимбабве'}],
  'text': 'Зимбабве'},
 {'text': ' '},
 {'analysis': [{'gr': 'V,пе=непрош,ед,изъяв,3-л,несов', 'lex': 'выдавать'}],
  'text': 'выдает'},
 {'text': ', '},
 {'analysis': [{'gr': 'CONJ=', 'lex': 'а'}], 'text': 'а'},
 {'text': ' '},
 {'analysis': [{'gr': 'PR=', 'lex': 'на'}], 'text': 'на'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,муж,неод=пр,ед', 'lex': 'сайт'}], 'text': 'сайте'},
 {'text': ' '},
 {'anal

Стоп-слова - незначащие слова, которые могут вносить помехи в классификатор, обычно их лучше вырезать

In [1]:
from nltk.corpus import stopwords

In [14]:
sw = stopwords.words('russian')

In [50]:
messages = [item.lower.split() for item in df.text]
words_flatten = [w for message in messages for w in message if w not in sw]

In [51]:
all_words = nltk.FreqDist(w.lower() for w in words_flatten)

In [52]:
word_features = [item[0] for item in all_words.most_common(500)]
featuresets = [(document_features(d), c) for (d,c) in zip(messages,df.is_female)]
train_set, test_set = featuresets[5000:], featuresets[:5000]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.9398
